In [1]:
# from https://github.com/lllyasviel/ControlNet/blob/main/docs/train.md

In [2]:
!nvidia-smi

Thu Feb  1 15:52:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
#%%capture
!pip install einops
!pip install omegaconf
!pip install open_clip_torch
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144574 sha256=860bff25fe456a41313c84ec93a71b66767e2cf41cb585708759b1ceaa3236a5
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.6
    Uninstalling wcwidth-0.2.6:
      Successfully uninstalled wcwidth-0.2.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 6.5

In [4]:
!rm -rf /kaggle/working/ControlNet

In [5]:
%cd /kaggle/working
#!git clone https://github.com/lllyasviel/ControlNet.git
!git clone https://github.com/kts12345/ControlNet.git

/kaggle/working
Cloning into 'ControlNet'...
remote: Enumerating objects: 1377, done.
remote: Counting objects: 100% (454/454), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 1377 (delta 405), reused 386 (delta 377), pack-reused 923
Receiving objects: 100% (1377/1377), 122.39 MiB | 30.22 MiB/s, done.
Resolving deltas: 100% (622/622), done.


In [6]:
#!wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt

## 加载control_sd15_ini.ckpt模型

In [7]:
# %cd /kaggle/working/ControlNet
# !ls -l

In [8]:
# !python tool_add_control.py ./models/v1-5-pruned.ckpt /kaggle/input/kts123s-controlnet/control_sd15_ini.ckpt

In [9]:
# # kaggle notebook 이 GPU 가속기 환경에서는 시스템 메모리가 13G 밖에 되지 않아 아래 명령어를 TPU 가속기로 바꾼 후 수행해야 한다.
# #!python tool_add_control.py ./models/v1-5-pruned.ckpt ./models/control_sd15_ini.ckpt
# # TPU 가속기로 바꿔서 생성한 weight 파일을 kaggle dataset 으로 올려 놨기 때문에 아래와 같이 copy 만 하면 된다.
# !ls -l ./models
# !cp /kaggle/input/kts123s-controlnet/control_sd15_ini.ckpt ./models
# !ls -l ./models

In [10]:
#  !ls -lh ./models/control_sd15_ini.ckpt

## 加载数据集

In [11]:
# !rm -rf /kaggle/working/ControlNet/training

In [12]:
!ls /kaggle/input/reference-controlnet-dataset-mid-3/

reference_controlnet_dataset


In [13]:
!mkdir /kaggle/working/ControlNet/training

In [14]:
!cp -r /kaggle/input/reference-controlnet-dataset-mid-3/reference_controlnet_dataset/* /kaggle/working/ControlNet/training
!ls /kaggle/working/ControlNet/training

build_datasets.py  original_videos_train  original_videos_val


In [15]:
# !pip install googletrans==4.0.0rc1

In [16]:
%cd /kaggle/working/ControlNet/training
!python /kaggle/working/ControlNet/training/build_datasets.py

/kaggle/working/ControlNet/training
515 frames extracted and saved to 'my_dataset_train/video_1'.
76 frames extracted and saved to 'my_dataset_train/video_2'.
2157 frames extracted and saved to 'my_dataset_train/video_3'.
30 frames extracted and saved to 'my_dataset_train/video_4'.
23 frames extracted and saved to 'my_dataset_train/video_5'.
20 frames extracted and saved to 'my_dataset_train/video_6'.
123 frames extracted and saved to 'my_dataset_train/video_7'.
83 frames extracted and saved to 'my_dataset_train/video_8'.
19 frames extracted and saved to 'my_dataset_train/video_9'.
15 frames extracted and saved to 'my_dataset_train/video_10'.
25 frames extracted and saved to 'my_dataset_train/video_11'.
300 frames extracted and saved to 'my_dataset_train/video_12'.
46 frames extracted and saved to 'my_dataset_train/video_13'.
27 frames extracted and saved to 'my_dataset_train/video_14'.
595 frames extracted and saved to 'my_dataset_train/video_15'.
25 frames extracted and saved to 'my_

## 开始训练

In [17]:
%cd /kaggle/working/ControlNet 
!ls /kaggle/working/ControlNet

/kaggle/working/ControlNet
LICENSE			       gradio_pose2image.py
README.md		       gradio_scribble2image.py
annotator		       gradio_scribble2image_interactive.py
cldm			       gradio_seg2image.py
config.py		       ldm
docs			       models
environment.yaml	       share.py
font			       test_imgs
github_page		       tool_add_control.py
gradio_annotator.py	       tool_add_control_sd21.py
gradio_canny2image.py	       tool_transfer_control.py
gradio_depth2image.py	       training
gradio_fake_scribble2image.py  tutorial_dataset.py
gradio_hed2image.py	       tutorial_dataset_test.py
gradio_hough2image.py	       tutorial_train.py
gradio_normal2image.py	       tutorial_train_sd21.py


In [18]:
import json
import cv2
import numpy as np

from torch.utils.data import Dataset

# dataset_path = './training/dataset_name/'

class MyDataset(Dataset):
    def __init__(self, dataset_path):
        self.data = []
        self.dataset_path = dataset_path
        with open(self.dataset_path + 'prompt.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']

        source = cv2.imread(self.dataset_path + source_filename)
        target = cv2.imread(self.dataset_path + target_filename)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)
    

In [19]:
!sed -i 's/ddim_steps=50/ddim_steps=20/g' /kaggle/working/ControlNet/cldm/cldm.py

In [20]:
import torch
import pytorch_lightning as pl
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict
from torch.utils.data import DataLoader

# Configs
#resume_path = '/kaggle/input/kts123s-controlnet/control_sd15_ini.ckpt'
resume_path = '/kaggle/input/controlnet1/epoch12-step22210.ckpt'
batch_size = 1
logger_freq = 500
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v15.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control

# Misc
dataset_train_path = './training/my_dataset_train/'
dataset_train = MyDataset(dataset_train_path)
dataloader_train = DataLoader(dataset_train, num_workers=4, batch_size=batch_size, shuffle=True)

dataset_val_path = './training/my_dataset_val/'
dataset_val = MyDataset(dataset_val_path)
dataloader_val = DataLoader(dataset_val, num_workers=4, batch_size=batch_size, shuffle=True)

logger = ImageLogger(batch_frequency=logger_freq)

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS

ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.14.mlp.fc1.weight', 'vision_model.post_layernorm.weight', 'vision_model.encoder.layers.6.self_attn.q_proj.weight', 'vision_model.encoder.layers.15.self_attn.k_proj.weight', 'vision_model.encoder.layers.20.self_attn.v_proj.bias', 'vision_model.encoder.layers.17.layer_norm2.weight', 'vision_model.encoder.layers.12.self_attn.k_proj.weight', 'vision_model.encoder.layers.11.self_attn.q_proj.weight', 'vision_model.encoder.layers.22.mlp.fc1.weight', 'vision_model.encoder.layers.21.mlp.fc1.bias', 'vision_model.encoder.layers.19.self_attn.k_proj.bias', 'vision_model.encoder.layers.1.mlp.fc2.weight', 'vision_model.encoder.layers.10.self_attn.out_proj.weight', 'vision_model.encoder.layers.23.self_attn.out_proj.bias', 'vision_model.encoder.layers.0.mlp.fc1.bias', 'vision_model.encoder.layers.2.self_attn.q_proj.weight', 'vision_model.encoder.layers.15.ml

Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is 768 and using 8 heads.
Setting up Me

In [21]:
# trainer = pl.Trainer(max_epochs=10, accelerator="gpu",
#                      precision='16-mixed', callbacks=[logger],
#                      accumulate_grad_batches=4,devices=2)
trainer = pl.Trainer(max_epochs=5, accelerator="gpu",
                     precision='16-mixed', callbacks=[logger],
                     accumulate_grad_batches=4,devices=1)
trainer.fit(model, dataloader_train, dataloader_val)

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('global_step', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


...

Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps



DDIM Sampler: 100%|██████████| 20/20 [00:13<00:00,  1.52it/s]


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 20 timesteps



DDIM Sampler: 100%|██████████| 20/20 [00:13<00:00,  1.52it/s]


Validation: 0it [00:00, ?it/s]

In [22]:
!du -h --max-depth=1 | sort -hr

9.5G	.
8.1G	./lightning_logs
1.2G	./training
123M	./.git
85M	./github_page
53M	./image_log
13M	./test_imgs
2.8M	./annotator
1.2M	./ldm
744K	./font
80K	./cldm
32K	./docs
12K	./models
